
#DatasetsTitanic

Extraccion de los datos y una breve ingenieria de los mismos

In [ ]:
import numpy as np
import pandas as pd


## Adquisicion de datos

Tendremos dos datasets un train y test ya predeterminados

In [ ]:
#obtencion de informacion de dataset
#train
df_train = pd.read_csv("https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv")

#test
df_test = pd.read_csv("https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/test.csv")
df_submission = pd.read_csv('https://raw.githubusercontent.com/mathias-brandewinder/Kaggle-Titanic/master/Titanic/Data/submission.csv', names=['Survived'])
df_test = pd.concat([df_test, df_submission], axis=1)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Manipulacion de datos

REvision leve de datos nulos y trasnformaciones de los mismos

In [ ]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

### Datos nulos

Como tenemos dos datasets , train y test ya definidos. Lo que debemos hacer es trabjar siempre con el train haciendo todo el llenado de datos como generacion de nuevas variables

Primeros trabajaremos con los datos del train y en base a esos valores , luego llenaremos la informacion en el test set

#### Train Datasets



In [ ]:
#llenados de nulos

#emabarked

#obtencion de la moda de la variable
train_embarked_mode = df_train['Embarked'].mode()[0]
df_train.Embarked = df_train.Embarked.fillna(train_embarked_mode)



In [ ]:
df_train['Embarked'].mode()[0]

'S'

In [ ]:
# fill age con la mediana de la agrupacion Sexo y Class

#creo datarrame de referencia
age_ref_null = df_train.groupby(['Sex', 'Pclass']).median()['Age'].reset_index()

#funcion de replace en base a dataframe agrupado
def fill_na_age_grouped(df_grouped , row_replace):
  if pd.isna(row_replace['Age']):
    return df_grouped[(df_grouped['Sex'] == row_replace['Sex']) & (df_grouped['Pclass'] == row_replace['Pclass'])]['Age'].values[0]
  else:
    return row_replace['Age']


#transformacion de train
df_train['Age'] = df_train.apply(lambda row: fill_na_age_grouped(age_ref_null, row), axis=1)


In [ ]:
age_ref_null

,Sex,Pclass,Age
0,female,1,35.0
1,female,2,28.0
2,female,3,21.5
3,male,1,40.0
4,male,2,30.0
5,male,3,25.0


In [ ]:
age_ref_null[(age_ref_null['Sex'] == 'female') & (age_ref_null['Pclass'] == 2)]['Age']

1    28.0
Name: Age, dtype: float64

In [ ]:
#fill CAbin con desconocido
df_train['Cabin'].fillna('desconocido', inplace=True)


In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,desconocido,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,desconocido,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,desconocido,S


### Preprocesamiento de variables categoricas a numericas

En este ejemplo tenemos variables como Name,  Sex,  Ticket ,  Cabin y Embarked como variables con string

Veremos que vamos hacer con algunas de ellas.

En algunos casos vamos a borrar algunas de las variables y ademas vamos a realizar un conocido procesamiento que se llama One Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#generate one hot encoder class
encoder = OneHotEncoder(handle_unknown='ignore', #ponemos ignore para poder ommitir cuando puede pasar que no exista los mismos valores
                        sparse=False) # generarlo como. matrix para poder generar las columnas adecuadas

#entrenamiento para genera el preprocesamiento one hot encoding para las variables indicadas
encoder.fit(df_train[['Sex', 'Embarked']])

#transformo en train datasets en base a la clase generada
cat_encoding = pd.DataFrame(encoder.transform(df_train[['Sex', 'Embarked']]), columns=encoder.get_feature_names_out())
cat_encoding.head()


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


In [ ]:
encoder.get_feature_names_out()

array(['Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype=object)

In [ ]:
#joineo de las variables que deseo con categoricas encoding
df_all_train = pd.concat([df_train, cat_encoding], axis=1)


df_all_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,desconocido,S,0.0,1.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0,1.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,desconocido,S,1.0,0.0,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0.0,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,desconocido,S,0.0,1.0,0.0,0.0,1.0


In [ ]:
#Borrado de columnas no necesarias
df_all_train.drop(['Name', 'Sex', 'Ticket', 'PassengerId', 'Cabin', 'Embarked'],axis=1, inplace=True)

#train datasets final
df_all_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [ ]:
#spliteo de datos en target y features for test and train

#seleccion de x values sin el target
X_train = df_all_train.loc[:,df_all_train.columns != 'Survived']

#seleccion del target
y_train = df_all_train.Survived


Transformacion del Test

Se debe mantener los mismo criterios que se hizo en el train

In [ ]:
#trasnformacion de test

#Embarked
df_test.Embarked = df_test.Embarked.fillna(train_embarked_mode)

#age
df_test['Age'] = df_test.apply(lambda row: fill_na_age_grouped(age_ref_null, row), axis=1)

#cabin
df_test['Cabin'].fillna('desconocido', inplace=True)

#llenado de Fare , en el train no existia
median_train_fare = df_train.Fare.median()
df_test.Fare.fillna(median_train_fare, inplace=True)

#one hot encoding
cat_encoding_test = pd.DataFrame(encoder.transform(df_test[['Sex', 'Embarked']]), columns=encoder.get_feature_names_out())
df_all_test = pd.concat([df_test, cat_encoding_test], axis=1)

#borrado de las mismas variables
df_all_test.drop(['Name', 'Sex', 'Ticket', 'PassengerId', 'Cabin', 'Embarked'],axis=1, inplace=True)



In [ ]:
# spliteo de datos y target del test
X_test = df_all_test.loc[:,df_all_test.columns != 'Survived']
y_test = df_all_test.Survived

